Global setup


In [1]:
n=4
k=2
q=23 #AA
eta=1
R.<x> = ZZ[]

Bob's setup

In [2]:
Ax = matrix([[randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q), randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q)], 
[randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q), randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q)]])
#Ax = matrix([[      4*x^3 + 4*x^2 + 10*x, 11*x^3 + 15*x^2 + 10*x + 3],
#[12*x^3 + 22*x^2 + 4*x + 12,             6*x^3 + x + 11]])
Ax  #public

[ 20*x^3 + 18*x^2 + 2*x + 5 17*x^3 + 2*x^2 + 11*x + 14]
[         8*x^2 + 19*x + 20       7*x^3 + 9*x^2 + 10*x]

In [43]:
latex(Ax)

\left(\begin{array}{rr}
20x^{3} + 18x^{2} + 2x + 5 & 17x^{3} + 2x^{2} + 11x + 14 \\
8x^{2} + 19x + 20 & 7x^{3} + 9x^{2} + 10x
\end{array}\right)

In [3]:
def binomial_centered():
    a1=randrange(0,2)
    a2=randrange(0,2)
    b1=randrange(0,2)
    b2=randrange(0,2)
#    return a1+a2-b1-b2
    return a1-b1
    return 0

In [4]:
sx = matrix([[binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()], 
            [binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()]])
#sx = matrix([[-x^3+x^2+1], [-x^2]])
sx  #private

[   -x^2]
[x^3 - x]

In [5]:
ex = matrix([[binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()], 
            [binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()]])
#ex = matrix([[x^2-x-1], [-x+1]])
ex #private

[-x^3 + 1]
[     x^2]

In [7]:
tx = matrix(((Ax*sx + ex) % (x^n+1) % q))
tx

[ 9*x^3 + 13*x^2 + 4*x + 2]
[18*x^3 + 10*x^2 - 9*x + 5]

In [44]:
latex(tx)

\left(\begin{array}{r}
9x^{3} + 13x^{2} + 4x + 2 \\
18x^{3} + 10x^{2} - 9x + 5
\end{array}\right)

Alice encodes her plaintext


In [62]:
plist = (ord("N") - ord("A") + 1).binary()
px = R(list(reversed(plist)))
px

x^3 + x^2 + x

In [46]:
rx = matrix([[binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()], 
            [binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()]])
rx

[-x + 1]
[     x]

In [47]:
e1 = matrix([[binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()],
             [binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()]])
e1

[          -x^2 - x]
[-x^3 + x^2 - x - 1]

In [48]:
latex(e1)

\left(\begin{array}{r}
-x^{2} - x \\
-x^{3} + x^{2} - x - 1
\end{array}\right)

In [49]:
e2 = binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()
e2

x^3

In [51]:
u = (Ax.transpose()*rx+e1) % (x^n+1) % q
u

[10*x^3 + 11*x^2 + 16*x + 2]
[               2*x^2 - 4*x]

In [52]:
latex(u)

\left(\begin{array}{r}
10x^{3} + 11x^{2} + 16x + 2 \\
2x^{2} - 4x
\end{array}\right)

In [53]:
tx.transpose()*rx % (x^n+1) % q

[6*x^3 + 7*x - 7]

In [55]:
round(q/2)*px

12*x^3 + 12*x^2 + 12*x

In [58]:
v = (tx.transpose()*rx+e2 + round(q/2)*px) % (x^n+1) % q
v

[19*x^3 + 12*x^2 + 19*x - 7]

In [59]:
latex(v)

\left(\begin{array}{r}
19x^{3} + 12x^{2} + 19x - 7
\end{array}\right)

$\mathbf{u}$ and $v$ is the ciphertext that Alice sends to Bob



In [63]:
(v-sx.transpose()*u) % (x^n+1) % q

[14*x^3 + 10*x^2 + 11*x - 22]

In [64]:
(round(q/2)^(-1)*((v-sx.transpose()*u) % (x^n+1) % q))

[7/6*x^3 + 5/6*x^2 + 11/12*x - 11/6]

In [65]:
pprime = (round(q/2)^(-1)*((v-sx.transpose()*u) % (x^n+1) % q))[0,0].map_coefficients(round, new_base_ring=Zmod(2)) ###UGH***
pprime

x^3 + x^2 + x

In [67]:
px

x^3 + x^2 + x

In [69]:
px == pprime

True

**Matrices for lattice attack**


In [13]:
matrix.options.max_rows(49) 

In [8]:
def negacirculant(v):
    Nshift = matrix([[0, 0, 0, -1],             #Negacyclic shift matrix
                     [1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 1, 0]])
    return sum([v[i]*Nshift^i for i in range(0, len(v))])

In [9]:
Ax

[ 20*x^3 + 18*x^2 + 2*x + 5 17*x^3 + 2*x^2 + 11*x + 14]
[         8*x^2 + 19*x + 20       7*x^3 + 9*x^2 + 10*x]

In [10]:
A = block_matrix([[negacirculant(Ax[0,0].padded_list(4)), negacirculant(Ax[0,1].padded_list(4))], 
                  [negacirculant(Ax[1,0].padded_list(4)), negacirculant(Ax[1,1].padded_list(4))]])
A

[  5 -20 -18  -2| 14 -17  -2 -11]
[  2   5 -20 -18| 11  14 -17  -2]
[ 18   2   5 -20|  2  11  14 -17]
[ 20  18   2   5| 17   2  11  14]
[---------------+---------------]
[ 20   0  -8 -19|  0  -7  -9 -10]
[ 19  20   0  -8| 10   0  -7  -9]
[  8  19  20   0|  9  10   0  -7]
[  0   8  19  20|  7   9  10   0]

In [11]:
latex(A)

\left(\begin{array}{rrrr|rrrr}
5 & -20 & -18 & -2 & 14 & -17 & -2 & -11 \\
2 & 5 & -20 & -18 & 11 & 14 & -17 & -2 \\
18 & 2 & 5 & -20 & 2 & 11 & 14 & -17 \\
20 & 18 & 2 & 5 & 17 & 2 & 11 & 14 \\
\hline
 20 & 0 & -8 & -19 & 0 & -7 & -9 & -10 \\
19 & 20 & 0 & -8 & 10 & 0 & -7 & -9 \\
8 & 19 & 20 & 0 & 9 & 10 & 0 & -7 \\
0 & 8 & 19 & 20 & 7 & 9 & 10 & 0
\end{array}\right)

In [12]:
tx

[ 9*x^3 + 13*x^2 + 4*x + 2]
[18*x^3 + 10*x^2 - 9*x + 5]

In [13]:
t = block_matrix([[column_matrix(1, 4, tx[0,0].padded_list(4))],
                [column_matrix(1, 4, tx[1,0].padded_list(4))]])
t

[ 2]
[ 4]
[13]
[ 9]
[--]
[ 5]
[-9]
[10]
[18]

In [14]:
latex(t)

\left(\begin{array}{r}
2 \\
4 \\
13 \\
9 \\
\hline
 5 \\
-9 \\
10 \\
18
\end{array}\right)

In [15]:
I8 = identity_matrix(8)
I8

[1 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0]
[0 0 0 1 0 0 0 0]
[0 0 0 0 1 0 0 0]
[0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 1]

In [16]:
M = block_matrix([[A.transpose()], [I8], [-t.transpose()], [q*I8]])
M

25 x 8 dense matrix over Integer Ring (use the '.str()' method to see the entries)

In [17]:
latex(M)

\left(\begin{array}{rrrrrrrr}
5 & 2 & 18 & 20 & 20 & 19 & 8 & 0 \\
-20 & 5 & 2 & 18 & 0 & 20 & 19 & 8 \\
-18 & -20 & 5 & 2 & -8 & 0 & 20 & 19 \\
-2 & -18 & -20 & 5 & -19 & -8 & 0 & 20 \\
14 & 11 & 2 & 17 & 0 & 10 & 9 & 7 \\
-17 & 14 & 11 & 2 & -7 & 0 & 10 & 9 \\
-2 & -17 & 14 & 11 & -9 & -7 & 0 & 10 \\
-11 & -2 & -17 & 14 & -10 & -9 & -7 & 0 \\
\hline
 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
\hline
 -2 & -4 & -13 & -9 & -5 & 9 & -10 & -18 \\
\hline
 23 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 23 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 23 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 23 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 23 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 23 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 23 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 23
\end{array}\right)

**Idea for lattice attack**


$$A(x) \star \mathbf{s}(x) + \mathbf{e}(x) - \mathbf{t}(x) \equiv \mathbf{0} \pmod{q}$$



so $$A \mathbf{s} + \mathbf{e} - \mathbf{t} \equiv \mathbf{0} \pmod{q}$$ for suitable matrices/vectors


or in block row form, $$(\mathbf{s}^T | \mathbf{e}^T | 1 | \mathbf{s'}^T)\begin{pmatrix} A^T \\ - \\ I \\ - \\ -\mathbf{t}^T \\ - \\ qI \end{pmatrix} =  \mathbf{s''}M = \mathbf{0} $$

so $\mathbf{s''}$ is a short vector in the kernel of $M$.

In [18]:
sT = block_matrix([[matrix(1, 4, sx[0,0].padded_list(4)), matrix(1, 4, sx[1,0].padded_list(4))]])
sT

[ 0  0 -1  0| 0 -1  0  1]

In [19]:
eT = block_matrix([[matrix(1, 4, ex[0,0].padded_list(4)), matrix(1, 4, ex[1,0].padded_list(4))]])
eT

[ 1  0  0 -1| 0  0  1  0]

In [20]:
spp = block_matrix([[s, e, matrix([1]), matrix([[0,0,0,0,0,0,0,0]])]])
spp

NameError: name 's' is not defined

In [ ]:
spp*M

In [ ]:
spp = block_matrix([[s, e, matrix([1]), matrix([[-2,-2,0,2,-2,-2,0,2]])]])
spp

In [ ]:
spp*M

In [ ]:
M.transpose().right_kernel_matrix(basis='LLL')  #There no left_kernel_matrix command for some reason

**Eve's attack**

Eve knows $A$ so she can set up $M$:

In [21]:
M

25 x 8 dense matrix over Integer Ring (use the '.str()' method to see the entries)

In [22]:
H, U = M.hermite_form(transformation = true)
U  #The transformation matrix that puts M into a (not very interesting) Hermite Normal Form

25 x 25 dense matrix over Integer Ring (use the '.str()' method to see the entries)

In [23]:
N = U[8:25]
N  #This should give a basis of the left kernel

[             113613104               77187852              -41424512               59557214             -282546813              154274337              -43222271               78115119             7700305610                      0                      0                      0                      0                      0                      0                      0                      0                      0                      0                      0                      0                      0                      0                      0                      0]
[   -263895057986456439    -179288233154777794      96218865706729222    -138336634496332353     656286158685739842    -358341015913420710     100394613915762169    -181442044380056240  -17885899811912416555                      5                      0                      0                      0                      0                      0                      0                      0                      0         

In [24]:
N.LLL()

[ 0  0 -1  0  0 -1  0  1  1  0  0 -1  0  0  1  0  1 -1  0  2  0  0  0  2  2]
[ 1  2 -2  0 -1  0  1  1  1 -3  3  0  1 -1  0  1 -1  1 -1 -1 -3 -1 -1  0  0]
[ 1  1 -3  2  0  1  0  0  5  1  1  2  1  0  0  0  0 -1 -2  1 -2  0 -1  1  0]
[ 0 -1  2 -3  0 -1  0  1  0 -1  3 -1 -2  0 -1  3 -2  0  0 -3  0 -2  1 -1  0]
[-2  0  1 -2  0 -2 -1  1 -1  1 -2  3 -3  1  0  3  0  0  0  2  2  0  1  1  2]
[ 0 -1  1 -2  2  0 -1  1  0  2 -3 -2 -1  0 -2 -3  1 -1 -2  0  0 -1 -1  0  2]
[ 2  1  0 -1 -2  0  0  1  3  1 -1 -1  2  0  0 -2  1  2  0 -1 -1 -2 -2  0  2]
[ 0  1  0 -1  0  1 -1  1  2  2  1  0 -1  2 -2  3  2  2 -2  1  0  0 -2  0  2]
[ 2 -1  0  3  0  1 -1 -1  1 -5 -1  0  0 -1 -1  0 -1 -1  1  0 -1  0  0 -1 -3]
[-3  2 -1  0 -1 -1  0  1 -3 -1  3 -1  4 -1 -1  1 -1  2  0  3  1  2  2  1  0]
[-2  3  0  1  1 -1 -2 -1  3 -3  1  2  1 -1 -1  1  0  1 -1  3  0  1 -2 -2 -1]
[-1  2 -2  0  1 -1  0 -2  2 -3 -1  1  0 -3 -3  1  0 -2 -2  0  0 -1 -2  0  1]
[ 3 -2  1  1  0  0  0  1 -1 -2  5  1  0  0  1  0  0 -1  2 -1 -2 -1  0  0 -1]

**Eve's attack showing calculation of transformation matrix**


In [25]:
M

25 x 8 dense matrix over Integer Ring (use the '.str()' method to see the entries)

In [26]:
latex(identity_matrix(25))

\left(\begin{array}{rrrrrrrrrrrrrrrrrrrrrrrrr}
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 

In [27]:
M1 = block_matrix([[M, identity_matrix(25)]])
M1

25 x 33 dense matrix over Integer Ring (use the '.str()' method to see the entries)

In [28]:
latex(M1)

\left(\begin{array}{rrrrrrrr|rrrrrrrrrrrrrrrrrrrrrrrrr}
5 & 2 & 18 & 20 & 20 & 19 & 8 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
-20 & 5 & 2 & 18 & 0 & 20 & 19 & 8 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
-18 & -20 & 5 & 2 & -8 & 0 & 20 & 19 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
-2 & -18 & -20 & 5 & -19 & -8 & 0 & 20 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
14 & 11 & 2 & 17 & 0 & 10 & 9 & 7 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
-17 & 14 & 11 & 2 & -7 & 0 & 10 & 9 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
-2 & -17 & 14 & 11 & -9 & -7 & 0 & 10 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 &

In [29]:
sage_input(M1)

matrix(ZZ, [[5, 2, 18, 20, 20, 19, 8, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [-20, 5, 2, 18, 0, 20, 19, 8, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [-18, -20, 5, 2, -8, 0, 20, 19, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [-2, -18, -20, 5, -19, -8, 0, 20, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [14, 11, 2, 17, 0, 10, 9, 7, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [-17, 14, 11, 2, -7, 0, 10, 9, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [-2, -17, 14, 11, -9, -7, 0, 10, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [-11, -2, -17, 14, -10, -9, -7, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0

In [30]:
H1 = M1.hermite_form()
H1 # The right side should be a transformation matrix (after some extra unnecessary but not harmful reduction)

25 x 33 dense matrix over Integer Ring (use the '.str()' method to see the entries)

In [31]:
latex(H1)

\left(\begin{array}{rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr}
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 21 & 5 & 7 & 9 & 16 & 18 & 14 & 11 & 1 & 1 & 6 & 4 & 2 & -5 & 4 & 8 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 

In [32]:
M.extended_echelon_form()

25 x 33 dense matrix over Integer Ring (use the '.str()' method to see the entries)

In [33]:
N1 = H1.submatrix(8, 8)
N1 # A basis for the kernel

[ 1  0  0  0  0  0  0  0  0  8  3 14  4 16 17 22 14  1  2  7  4  2 -7  5 10]
[ 0  1  0  0  0  0  0  0  0  1  6  7 19  1 19 19 13  2  2  7  4  2 -6  4  9]
[ 0  0  1  0  0  0  0  0  0  7 16  9  9 12  5  3 14  2  3  7  5  3 -6  5 10]
[ 0  0  0  1  0  0  0  0  0 14  7  9 14 17 13  8 22  2  4 13  8  5 -9  9 16]
[ 0  0  0  0  1  0  0  0  0 17 20  0 12 19 15  4  7  0  0  3  2  1 -4  2  5]
[ 0  0  0  0  0  1  0  0  0 21  5  2 22 19 20 22  3  1 -1  1  1  0 -2  0  1]
[ 0  0  0  0  0  0  1  0  0 13 19  3  4 16 13 18 22  2  4 11  8  5 -9  9 16]
[ 0  0  0  0  0  0  0  1  0  3  3 17 17  1 21 16  6  1  1  4  1  1 -2  2  4]
[ 0  0  0  0  0  0  0  0  1  2 18 16 14  7  5  9 12  1  2  6  4  2 -5  5  9]
[ 0  0  0  0  0  0  0  0  0 23  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0 23  0  0  0  0  0  0  0  0 -1  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0 23  0  0  0  0  0  0  0  0 -1  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0 23  0  0  0  0  0  0  0  0 -1  0  0  0]

In [34]:
latex(N1)

\left(\begin{array}{rrrrrrrrrrrrrrrrrrrrrrrrr}
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 8 & 3 & 14 & 4 & 16 & 17 & 22 & 14 & 1 & 2 & 7 & 4 & 2 & -7 & 5 & 10 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 6 & 7 & 19 & 1 & 19 & 19 & 13 & 2 & 2 & 7 & 4 & 2 & -6 & 4 & 9 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 7 & 16 & 9 & 9 & 12 & 5 & 3 & 14 & 2 & 3 & 7 & 5 & 3 & -6 & 5 & 10 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 14 & 7 & 9 & 14 & 17 & 13 & 8 & 22 & 2 & 4 & 13 & 8 & 5 & -9 & 9 & 16 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 17 & 20 & 0 & 12 & 19 & 15 & 4 & 7 & 0 & 0 & 3 & 2 & 1 & -4 & 2 & 5 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 21 & 5 & 2 & 22 & 19 & 20 & 22 & 3 & 1 & -1 & 1 & 1 & 0 & -2 & 0 & 1 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 13 & 19 & 3 & 4 & 16 & 13 & 18 & 22 & 2 & 4 & 11 & 8 & 5 & -9 & 9 & 16 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 3 & 3 & 17 & 17 & 1 & 21 & 16 & 6 & 1 & 1 & 4 & 1 & 1 & -2 & 2 & 4 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 2 & 18 & 16 & 14 & 7 & 5 & 9 & 12 & 1 & 2 & 6 & 4 & 2 & -

In [35]:
N1.LLL()

[ 0  0 -1  0  0 -1  0  1  1  0  0 -1  0  0  1  0  1 -1  0  2  0  0  0  2  2]
[ 0 -1  0  1  0 -1  1 -1 -2 -2 -1  0  1 -2  2 -3 -2 -2  2 -1  0  0  2  0 -2]
[ 0 -2  1 -1  1  0 -1  0  4  3  1  1 -2  1 -1  2 -1 -2 -1 -1  1 -1  1  0  0]
[-1  2 -1 -1 -2  1  1  2  0  2 -1 -2  1  2  2  0 -1  4 -1  0 -1  1  1  0  0]
[-2  4 -3 -1 -1  0  1  0  2 -1 -2 -1  1 -1 -1  1 -1  2 -3  0 -1  0 -1  0  1]
[ 1  0  1 -1  0  1  0  1 -1  0  2 -6  1  1 -1  0  3  2  0  0  0  0 -2  0  2]
[-2  0  0  0 -1  0  1 -2  2  1  0 -4  1  0  1  2 -1  0  1 -1  3  1  2  0 -1]
[-2 -1  0  1  2  0  0 -1 -3 -1  1  1 -2  0  0  2 -1 -2  0  1  1  2  2  0 -2]
[ 1  1 -3  2  0  1  0  0  5  1  1  2  1  0  0  0  0 -1 -2  1 -2  0 -1  1  0]
[ 1  2 -2  0 -1  0  1  1  1 -3  3  0  1 -1  0  1 -1  1 -1 -1 -3 -1 -1  0  0]
[-2  1  0 -3  0 -1  1  1 -1  5  1  0  0  1  1  0  1  1 -1  0  1  0  0  1  3]
[ 1 -1 -2 -1  0  0  2 -1 -1  0 -2 -1 -1 -2 -2  0  0 -3 -1 -3  0 -2  0  2  2]
[ 2  0  0  2 -1  1  2 -1 -1  0 -1 -2  0 -2 -4 -3  2  1  3 -1 -1  1 -1  0 -1]

In [36]:
latex(N1.LLL())

\left(\begin{array}{rrrrrrrrrrrrrrrrrrrrrrrrr}
0 & 0 & -1 & 0 & 0 & -1 & 0 & 1 & 1 & 0 & 0 & -1 & 0 & 0 & 1 & 0 & 1 & -1 & 0 & 2 & 0 & 0 & 0 & 2 & 2 \\
0 & -1 & 0 & 1 & 0 & -1 & 1 & -1 & -2 & -2 & -1 & 0 & 1 & -2 & 2 & -3 & -2 & -2 & 2 & -1 & 0 & 0 & 2 & 0 & -2 \\
0 & -2 & 1 & -1 & 1 & 0 & -1 & 0 & 4 & 3 & 1 & 1 & -2 & 1 & -1 & 2 & -1 & -2 & -1 & -1 & 1 & -1 & 1 & 0 & 0 \\
-1 & 2 & -1 & -1 & -2 & 1 & 1 & 2 & 0 & 2 & -1 & -2 & 1 & 2 & 2 & 0 & -1 & 4 & -1 & 0 & -1 & 1 & 1 & 0 & 0 \\
-2 & 4 & -3 & -1 & -1 & 0 & 1 & 0 & 2 & -1 & -2 & -1 & 1 & -1 & -1 & 1 & -1 & 2 & -3 & 0 & -1 & 0 & -1 & 0 & 1 \\
1 & 0 & 1 & -1 & 0 & 1 & 0 & 1 & -1 & 0 & 2 & -6 & 1 & 1 & -1 & 0 & 3 & 2 & 0 & 0 & 0 & 0 & -2 & 0 & 2 \\
-2 & 0 & 0 & 0 & -1 & 0 & 1 & -2 & 2 & 1 & 0 & -4 & 1 & 0 & 1 & 2 & -1 & 0 & 1 & -1 & 3 & 1 & 2 & 0 & -1 \\
-2 & -1 & 0 & 1 & 2 & 0 & 0 & -1 & -3 & -1 & 1 & 1 & -2 & 0 & 0 & 2 & -1 & -2 & 0 & 1 & 1 & 2 & 2 & 0 & -2 \\
1 & 1 & -3 & 2 & 0 & 1 & 0 & 0 & 5 & 1 & 1 & 2 & 1 & 0 & 0 & 0 & 0 & -1 & -2

In [41]:
N1.LLL()[0:1, 0:8]

[ 0  0 -1  0  0 -1  0  1]

In [38]:
sx

[   -x^2]
[x^3 - x]

In [40]:
block_matrix([[matrix(1, 4, sx[0,0].padded_list(4)),
                matrix(1, 4, sx[1,0].padded_list(4))]])


[ 0  0 -1  0| 0 -1  0  1]

The secret key is recovered.